In [4]:
from datasets import load_dataset
from datasets import Dataset
ds = load_dataset("Eladio/emrqa-msquad")

In [5]:
ds.save_to_disk("emrqa-msquad")

Saving the dataset (1/1 shards): 100%|██████████| 32739/32739 [00:00<00:00, 413894.36 examples/s]


In [ ]:
dataset = Dataset.from_file("emrqa-msquad/train/data-00000-of-00001.arrow")
dataset.to_csv("dataset/raw_data.csv")

Creating CSV from Arrow format: 100%|██████████| 131/131 [00:19<00:00,  6.60ba/s]


318327467

## Transfer to Json file with better format

In [13]:
import pandas as pd
import json
import re

def clean_answer_string(answers_str):
    # Bỏ dtype=object
    answers_str = re.sub(r",\s*dtype=object", "", answers_str)

    # Thay array([...]) -> [...]
    answers_str = re.sub(r"array\((\[.*?\])\)", r"\1", answers_str)

    # Bỏ nháy đơn ở giữa từ (dc'ing -> dcing, Ringer's -> Ringers)
    answers_str = re.sub(r"(\w)'(\w)", r"\1\2", answers_str)

    # Bỏ nháy đơn ở cuối từ (Graves' -> Graves)
    answers_str = re.sub(r"(\w)'(\s|\"|$)", r"\1\2", answers_str)

    # Cuối cùng thay nháy đơn -> nháy kép
    answers_str = answers_str.replace("'", '"')

    return json.loads(answers_str)


def convert_csv_to_squad_json_with_error_log(csv_path, output_json_path):
    df = pd.read_csv(csv_path)

    # Chỉ lấy 15 dòng đầu
    

    data = []
    error_rows = []

    for idx, row in df.iterrows():
        try:
            context = row["context"]
            question = row["question"]
            answers = clean_answer_string(row["answers"])

            # Kiểm tra định dạng
            if not all(k in answers for k in ["text", "answer_start"]):
                raise ValueError("Thiếu key trong answers")

            # Tạo sample
            sample = {
                "context": context,
                "question": question,
                "answers": {
                    "text": answers["text"],
                    "answer_start": answers["answer_start"]
                },
                "id": str(idx)
            }
            data.append(sample)
        except Exception as e:
            error_rows.append({"index": idx, "error": str(e), "raw": row["answers"]})
            print(f"❌ Lỗi ở dòng {idx}: {e}")
            print(f"   Nội dung answers: {row['answers']}\n")

    # Ghi JSON hợp lệ
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    # Báo lỗi nếu có
    if error_rows:
        print(f"\n⚠️ Tổng cộng {len(error_rows)} dòng bị lỗi:")
        for e in error_rows:
            print(f"- Dòng {e['index']}: {e['error']}")
            print(f"  Nội dung: {e['raw']}\n")
    else:
        print("✅ Không có lỗi nào.")

    print(f"\n✅ JSON hợp lệ đã được lưu vào: {output_json_path}")

# Chạy thử
convert_csv_to_squad_json_with_error_log("dataset/raw_data.csv", "dataset/cleaned_dataset.json")


✅ Không có lỗi nào.

✅ JSON hợp lệ đã được lưu vào: dataset/cleaned_dataset.json
